In [182]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
import statsmodels.api as sm

In [183]:
file_path = 'C:/Users/Abund/Downloads/bike_project/Bikeshare/Data/'
file_name = 'daily.csv'
df = pd.read_csv(file_path + file_name, index_col=0, header=0)

In [184]:
df.head()

,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
instant,,,,,,,,,,,,,,,
1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [185]:
# select X by ditching first column with dates
X = df.iloc[:,1:]

In [186]:
# select y by popping out count column
y = X.pop('cnt')

In [187]:
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=123)

In [188]:
y_train = y_train.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

In [189]:
sc_X = MinMaxScaler(feature_range=(0,1))
sc_y = MinMaxScaler(feature_range=(0,1))

sc_X.fit(X_train)
sc_y.fit(y_train)

y_train = sc_y.transform(y_train)
y_test = sc_y.transform(y_test)
X_train = sc_X.transform(X_train)
X_test = sc_X.transform(X_test)

In [190]:
# performing exhaustive search on my variables  using mean squared error
lr = LinearRegression()
efs = EFS(lr, min_features = 1, max_features = 13, scoring='neg_mean_squared_error',print_progress=True, cv=5)

In [191]:
# create efs fit
efs = efs.fit(X_train,y_train)

Features: 6114/8191IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Features: 8191/8191

In [193]:
print('best features:', efs.best_idx_)

best features: (5, 11, 12)


In [194]:
# subset best features for MLR
X_train_best = X_train[:,[5,11,12]]
X_test_best = X_test[:,[5,11,12]]

In [195]:
# create mlr
mlr = LinearRegression()
# fit mlr
mlr.fit(X_train_best,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [197]:
y_hat = sc_y.inverse_transform(mlr.predict(X_test_best))
y = sc_y.inverse_transform(y_test)


SS_Residual = sum((y-y_hat)**2)       
SS_Total = sum((y-np.mean(y))**2)     
r_squared = 1 - ((float(SS_Residual))/SS_Total)
print(f'''
Coefficients: {mlr.coef_}
Intercept:    {mlr.intercept_}
R^2 score:    {r_squared}
''')


Coefficients: [[-1.44877549e-16  3.99390601e-01  8.11672331e-01]]
Intercept:    [4.4408921e-16]
R^2 score:    [1.]

